## LLM Flow

#### Imports 

In [1]:
from langchain.prompts.chat import SystemMessagePromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.prompts.chat import AIMessagePromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chat_models import ChatVertexAI
from langchain.schema import SystemMessage
from langchain.schema import HumanMessage
from google.cloud import aiplatform
from google.cloud import bigquery
import pandas as pd
import langchain
import datetime
import logging
import google

##### Setup logging

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

#### Essentials 

In [3]:
%cd /home/jupyter/bq-sql-agent

/home/jupyter/bq-sql-agent


In [4]:
%pwd

'/home/jupyter/bq-sql-agent'

In [5]:
PROJECT = 'arun-genai-bb'
LOCATION = 'us-central1'
MODEL_NAME = 'chat-bison'

In [6]:
bq = bigquery.Client()

In [7]:
llm = ChatVertexAI(project=PROJECT, 
                   location=LOCATION, 
                   model_name=MODEL_NAME,
                   temperature=0.0)

### Step 1: Identify the `intent` of the user's query

In [11]:
prompt = "I would like to make a reservation in Miami for four nights from September 23rd to the 28th. What are my options?"

##### Load example prompt and completion pairs needed for intent detection

In [12]:
messages = []

In [13]:
examples = pd.read_csv('./DATA/hotel_reservation_prompts.csv')
examples.head()

,Prompt,Intent
0,I'm looking to book a single room in Orlando f...,MAKE_RESERVATION
1,Can I reserve a suite in New York from Decembe...,MAKE_RESERVATION
2,I'd like to change my reservation from a doubl...,MODIFY_RESERVATION
3,Is it possible to cancel my booking for the we...,CANCEL_RESERVATION
4,Do you have any available rooms with a sea vie...,CHECK_AVAILABILITY


In [14]:
template = "You are a helpful assistant that can detect intent from a user's query"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
messages.append(system_message_prompt)

In [15]:
for _, row in examples.iterrows():
    prompt, intent = row
    human_message = HumanMessagePromptTemplate.from_template(prompt)
    messages.append(human_message)
    intent_message = AIMessagePromptTemplate.from_template(intent)
    messages.append(intent_message)

In [16]:
messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful assistant that can detect intent from a user's query", template_format='f-string', validate_template=True), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="I'm looking to book a single room in Orlando for three nights starting October 5th.", template_format='f-string', validate_template=True), additional_kwargs={}),
 AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='MAKE_RESERVATION', template_format='f-string', validate_template=True), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Can I reserve a suite in New York from December 1st to the 5th?', template_format='f-string', validate_templ

In [17]:
human_template = "{user_query}"
human_message = HumanMessagePromptTemplate.from_template(human_template)
messages.append(human_message)

In [22]:
chat_prompt = ChatPromptTemplate.from_messages(messages)

In [23]:
user_query = "I would like to make a reservation in Miami for four nights from September 23rd to the 28th. What are my options?"
request = chat_prompt.format_prompt(user_query=user_query).to_messages()
logger.info(request)

[SystemMessage(content="You are a helpful assistant that can detect intent from a user's query", additional_kwargs={}), HumanMessage(content="I'm looking to book a single room in Orlando for three nights starting October 5th.", additional_kwargs={}, example=False), AIMessage(content='MAKE_RESERVATION', additional_kwargs={}, example=False), HumanMessage(content='Can I reserve a suite in New York from December 1st to the 5th?', additional_kwargs={}, example=False), AIMessage(content='MAKE_RESERVATION', additional_kwargs={}, example=False), HumanMessage(content="I'd like to change my reservation from a double room to a suite.", additional_kwargs={}, example=False), AIMessage(content='MODIFY_RESERVATION', additional_kwargs={}, example=False), HumanMessage(content='Is it possible to cancel my booking for the weekend?', additional_kwargs={}, example=False), AIMessage(content='CANCEL_RESERVATION', additional_kwargs={}, example=False), HumanMessage(content='Do you have any available rooms with

In [24]:
%%time 

response = llm(request)

CPU times: user 7.7 ms, sys: 7.47 ms, total: 15.2 ms
Wall time: 563 ms


In [25]:
logger.info(response.content.strip())

MAKE_RESERVATION
